<a href="https://colab.research.google.com/github/jimpea/mustached-batman/blob/master/pbc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PBC

Follow this [reference](https://python.plainenglish.io/molecular-dynamics-periodic-boundary-conditions-21f957bbb294) to practice implementation of periodic boundary conditions in Python. To start, use a simple one dimension, unit cell 0 < x <= 1. This has two images:

- -1 < x <= 0
- 1 < x <= 2

The cut-off radius must be set to less than half the unit cell

Set up a one dimensional PBC with five points in the unit cell at `[0, 0.2, 0.4, 0.6, 0.8]`.

> **Note:** Set a numpy `(m,n)` array to a `(m*n,1)` column vector using `a.reshape(-1, 1)`. See [StackOverflow](https://stackoverflow.com/questions/18691084/what-does-1-mean-in-numpy-reshape).

The images are then calculated as:

In [1]:
!pip install celluloid

In [2]:
import numpy as np
import random
from matplotlib import pyplot as plt
from celluloid import Camera

atom_x_pos = np.arange(0, 1, 0.2)
unit_cell_length = 1

for x_shift in [-1, 0, 1]:
    if x_shift == 0:
        continue
    else:
        image_atom_x_pos = atom_x_pos + unit_cell_length * x_shift
        print(image_atom_x_pos)


[-1.  -0.8 -0.6 -0.4 -0.2]
[1.  1.2 1.4 1.6 1.8]


In [3]:
import random
import numpy as np

random.seed(10)
numAtoms = 10
unitCellLength = 10

positions = np.random.rand(unitCellLength, 1) * unitCellLength + unitCellLength
velocities = (np.random.rand(unitCellLength, 1) - 0.5)

iterations = 600
timestep = 0.02

def check_boundaries(positions, unitCellLength):
    #Move positions back into the cell if exceed the cell boundary.
    for posVal in range(len(positions)):
        for col in [0]:
            if positions[posVal, col]  > unitCellLength * 2:
                positions[posVal, col] = positions[posVal, col] - unitCellLength
            if positions[posVal, col] < unitCellLength:
                positions[posVal, col] = positions[posVal, col] + unitCellLength

def update_positions(positions, velocities, timestep):
    #Update the positions based on the velocities
    positions += positions * velocities*timestep

def makeImages(positions, unitCellLength):
    numParts = len(positions)
    allParticles = np.zeros((numParts*3, 1))
    iter = -1
    while iter < numParts - 1:
        iter += 1
        allParticles[iter] = positions[iter]
    xmult = np.ones((numParts, 1))
    for xshift in range(1, -2, -1):
        if xshift == 0: continue
        else:
            tempParticles = positions + xmult*xshift*unitCellLength

            for i in range(numParts):
                iter += 1
                allParticles[iter] = tempParticles[i]
    return allParticles

r = makeImages(positions, unitCellLength)



In [ ]:
fig = plt.figure()
plt.xlim([0,unitCellLength*3])
plt.ylim([0,unitCellLength*3])
camera = Camera(fig)
#colorsList = random.choices(range(1,20), k = len(positions))
#colors = colorsList*3
colorsUnit = ['b'] * len(positions)
colorsOther = ['g'] * len(positions) * 2
colors = colorsUnit + colorsOther
for i in range(0, iterations):
    allAtoms = makeImages(positions, unitCellLength)
    plt.scatter(allAtoms[:,0], np.zeros(unitCellLength*3) + 10, c=colors, alpha=0.8)
    update_positions(positions, velocities, timestep)
    check_boundaries(positions, unitCellLength)
    camera.snap()

animation = camera.animate()
animation.save('pbc_tile_color.gif', writer='pillow', fps=40)




![pbc](/content/pbc_tile_color.gif)


![pbc](pbc_tile_color.gif)